#  RAG + Impression Grading System


In [1]:
# ============================================================
# 📦 Install Dependencies
# ============================================================
!pip install sentence-transformers transformers faiss-cpu spacy rich bitsandbytes accelerate --quiet
!python -m spacy download en_core_web_sm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 75.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
!pip install faiss-cpu

In [3]:
# ============================================================
# 1️⃣ Imports and Model Loading
# ============================================================
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM
import faiss, numpy as np, torch, spacy, re
from rich.console import Console
from rich.table import Table

console = Console()
console.print("[bold cyan]Loading models (please wait)...[/bold cyan]")

# Embedding model for retrieval
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
nlp = spacy.load("en_core_web_sm")

tok = AutoTokenizer.from_pretrained("prometheus-eval/prometheus-7b-v2.0")
impression_model = AutoModelForCausalLM.from_pretrained(
    "prometheus-eval/prometheus-7b-v2.0",
    device_map="auto",
    load_in_4bit=False,
    torch_dtype=torch.bfloat16
)

device = "cuda" if torch.cuda.is_available() else "cpu"
console.print(f"[green]✅ Models loaded successfully on {device.upper()}.[/green]")

Loading models (please wait)...

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00004-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/789M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

✅ Models loaded successfully on CUDA.

In [4]:
# ============================================================
# 2️⃣ Mini Knowledge Base for RAG
# ============================================================
corpus = [
    "Photosynthesis converts sunlight, carbon dioxide, and water into glucose and oxygen.",
    "Chlorophyll absorbs blue and red light and reflects green light.",
    "Plants use chloroplasts to capture solar energy during photosynthesis.",
    "Mitosis produces two genetically identical daughter cells."
]

embeddings = embedder.encode(corpus, normalize_embeddings=True)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(np.array(embeddings))
console.print("[green]✅ Knowledge base indexed for RAG retrieval.[/green]")


✅ Knowledge base indexed for RAG retrieval.

In [5]:
# ============================================================
# 3️⃣ RAG Context Retrieval
# ============================================================
def retrieve_context(ans, top_k=2):
    """Retrieve top-k relevant facts from the mini knowledge base."""
    ans_emb = embedder.encode([ans], normalize_embeddings=True)
    sim, idx = index.search(np.array(ans_emb), k=top_k)
    context = "\n".join([corpus[i] for i in idx[0]])
    return context


In [6]:
def impression_feedback(question, reference, ans, context):
    """
    Teacher-style evaluator that uses LLM for reasoning feedback,
    and SentenceTransformer similarity for fallback score.
    """
    # --- Step 1: Ask the model concisely ---
    prompt = f"""
You are a teacher grading a student's short written answer.

Question: {question}
Reference Answer: {reference}
Relevant Knowledge: {context}
Student Answer: {ans}

Evaluate how correct and complete the student's answer is.
Give a score (0–100) and one feedback sentence.
Format:
Score: <number>
Feedback: <short feedback>
    """.strip()

    inputs = tok(prompt, return_tensors="pt").to(device)
    output = impression_model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.5,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tok.eos_token_id
    )

    response = tok.decode(output[0], skip_special_tokens=True)
    response = re.sub(r"\s+", " ", response).strip()

    # --- Step 2: Extract score + feedback ---
    score_match = re.search(r"score[:\-]?\s*(\d{1,3})", response, re.I)
    feedback_match = re.search(r"feedback[:\-]?\s*(.*)", response, re.I)

    score = score_match.group(1) if score_match else None
    feedback = feedback_match.group(1).strip() if feedback_match else response

    # --- Step 3: If no numeric score, use semantic fallback ---
    if score is None:
        ref_emb = embedder.encode(reference, convert_to_tensor=True, normalize_embeddings=True)
        ans_emb = embedder.encode(ans, convert_to_tensor=True, normalize_embeddings=True)
        sem = float(util.cos_sim(ref_emb, ans_emb))
        score = int(sem * 100)
        feedback += f" (Estimated via similarity: {score})"

    return score, feedback


In [7]:
def evaluate(question, reference, students):
    """Run evaluation for multiple student answers."""
    table = Table(show_header=True, header_style="bold magenta")
    table.add_column("Student", style="cyan", width=20)
    table.add_column("Score", justify="center", width=8)
    table.add_column("Feedback", width=80)

    for sid, ans in students.items():
        context = retrieve_context(ans)
        score, feedback = impression_feedback(question, reference, ans, context)  # ✅ unpack tuple
        table.add_row(sid, str(score), feedback[:400])
    console.print(table)


In [8]:
# ============================================================
# 6️⃣ Demo Run
# ============================================================
question = "What is photosynthesis?"
reference = (
    "Photosynthesis is the process by which green plants convert sunlight, carbon dioxide, "
    "and water into glucose and oxygen."
)

students = {
    "A1 - short-accurate": "Plants make food using sunlight and carbon dioxide.",
    "A2 - long-detailed": "Green plants use chlorophyll in chloroplasts to absorb sunlight, "
                          "convert CO₂ and water into glucose and release oxygen – this process is photosynthesis.",
    "A3 - partial": "Plants need light and water to survive.",
    "A4 - off-topic": "Mitosis creates two identical cells for growth."
}

evaluate(question, reference, students)


┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Student             ┃  Score  ┃ Feedback                                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ A1 - short-accurate │   60    │ sentence. Format: Score: <number> Feedback: <short feedback> The student's      │
│                     │         │ answer is somewhat correct, but it lacks the completeness and detail of the     │
│                     │         │ reference answer. The student correctly identifies that plants use sunlight and │
│                     │         │ carbon dioxide to make food, but fails to mention the role of water and the     │
│                     │         │ production of oxygen. Additionally, the student does not mention the role of    │
│                     │         │ chloroplasts in                                                                 │
│ A2 - long-detailed  │   85    │ sentence. Format: Score: <number> Feedback: <short feedback> The student's      │
│                     │         │ answer correctly identifies the key components of photosynthesis: the use of    │
│                     │         │ chlorophyll in chloroplasts, the conversion of CO₂ and water into glucose, and  │
│                     │         │ the release of oxygen. However, the response could have been more comprehensive │
│                     │         │ by mentioning the role of sunlight in this process. The student's answer also   │
│                     │         │ lacks the s                                                                     │
│ A3 - partial        │   25    │ sentence. Format: Score: <number> Feedback: <short feedback> The student's      │
│                     │         │ answer is not entirely correct or complete. It mentions the need for light and  │
│                     │         │ water for plants to survive but fails to explain the process of photosynthesis. │
│                     │         │ It does not mention the conversion of sunlight, carbon dioxide, and water into  │
│                     │         │ glucose and oxygen, which is a key part of photosynthesis. Additionally, it     │
│                     │         │ does not me                                                                     │
│ A4 - off-topic      │   11    │ sentence. Format: Score: <number> Feedback: <short feedback> The student's      │
│                     │         │ answer is incorrect. It does not address the question about photosynthesis.     │
│                     │         │ Instead, it discusses mitosis, which is unrelated to the concept of             │
│                     │         │ photosynthesis. The student's answer lacks any relevant information about       │
│                     │         │ photosynthesis, such as the conversion of sunlight, carbon dioxide, and water   │
│                     │         │ into glucose and oxygen. Ther                                                   │
└─────────────────────┴─────────┴─────────────────────────────────────────────────────────────────────────────────┘